在进行数据分析时，经常需要根据一定条件创建新的数据列，然后及逆行进一步分析
1. 直接赋值
2. df.apply方法
3. df.assign方法
4. 按条件选择分组分别赋值

In [1]:
import pandas as pd
df=pd.read_excel("./weater_beijing.xlsx")
df.head()

,日期,最高温,最低温,天气,风力风向,空气质量指数
0,2011-01-01 周六,-2°,-7°,多云~阴,无持续风向微风,NaN
1,2011-01-02 周日,-2°,-7°,多云,无持续风向微风,NaN
2,2011-01-03 周一,-2°,-6°,多云~阴,西北风~北风3-4级~4-5级,NaN
3,2011-01-04 周二,-2°,-9°,晴,北风5-6级,NaN
4,2011-01-05 周三,-2°,-10°,晴,北风~无持续风向3-4级~微风,NaN


In [15]:
# 替换温度的后缀°
df.loc[:,'最低温']=df['最低温'].str.replace("°","")
df.loc[:,'最高温']=df['最高温'].str.replace("°","")

In [29]:
df.loc[:,'最低温'].fillna('0',inplace=True)

In [32]:
df.loc[df['最低温']=='']

,日期,最高温,最低温,天气,风力风向,空气质量指数
455,2012-04-11 周三,9,,多云~,无持续风向~微风~,NaN


In [34]:
# 将最低温中''替换成0
df.loc[:,'最低温']=df['最低温'].replace('','0')

In [36]:
# 将最高温和最低温列的obj类型转换成int型
df.loc[:,'最低温']=df.loc[:,'最低温'].astype('int32')
df.loc[:,'最高温']=df.loc[:,'最高温'].astype('int32')

C:\Users\JDQ\AppData\Local\Temp\ipykernel_17304\77100508.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'最高温']=df.loc[:,'最高温'].astype('int32')


In [30]:
df.loc[df['日期']=='2012-04-11 周三']

,日期,最高温,最低温,天气,风力风向,空气质量指数
455,2012-04-11 周三,9,,多云~,无持续风向~微风~,NaN


1. 直接赋值

实例：计算温差

In [42]:
df.dtypes

日期        object
最高温        int32
最低温        int32
天气        object
风力风向      object
空气质量指数    object
dtype: object

In [43]:
df['wencha']=df['最高温']-df['最低温']

In [46]:
df.head()

,日期,最高温,最低温,天气,风力风向,空气质量指数,wencha
0,2011-01-01 周六,-2,-7,多云~阴,无持续风向微风,NaN,5
1,2011-01-02 周日,-2,-7,多云,无持续风向微风,NaN,5
2,2011-01-03 周一,-2,-6,多云~阴,西北风~北风3-4级~4-5级,NaN,4
3,2011-01-04 周二,-2,-9,晴,北风5-6级,NaN,7
4,2011-01-05 周三,-2,-10,晴,北风~无持续风向3-4级~微风,NaN,8


2. df.apply方法
DataFrame.apply(func, axis=0, raw=False, result_type=None, args=(), **kwargs)[source]
Apply a function along an axis of the DataFrame.

Objects passed to the function are Series objects whose index is either the DataFrame’s index (axis=0) or the DataFrame’s columns (axis=1). By default (result_type=None), the final return type is inferred from the return type of the applied function. Otherwise, it depends on the result_type argument.

实例：添加一列温度类型
    1. 如果最高温度大于33度就是高温
    2. 如果小于-10度就是低温
    3. 否则就是常温

In [49]:
def get_wendu_type(x):
    if x['最高温']>33:
        return '高温'
    if x['最低温']<-10:
        return '低温'
    return '常温'

# 这里需要设置axis==1，这是series的index是columns
df.loc[:,'wendu_type']=df.apply(get_wendu_type,axis=1)

In [50]:
# 产看温度类型的计数
df['wendu_type'].value_counts()

常温    3730
高温     231
低温      45
Name: wendu_type, dtype: int64

3. df.assign方法
DataFrame.assign(**kwargs)[source]
Assign new columns to a DataFrame.

Returns a new object with all original columns in addition to new ones. Existing columns that are re-assigned will be overwritten.

实例：将温度从摄氏度变为华氏度

In [51]:
# 可以同时添加多个新的列
df.assign(
    华氏度最高温=lambda x:x['最高温']*9/5+32,
    华氏度最低温=lambda x:x['最低温']*9/5+32,
)

,日期,最高温,最低温,天气,风力风向,空气质量指数,wencha,wendu_type,华氏度最高温,华氏度最低温
0,2011-01-01 周六,-2,-7,多云~阴,无持续风向微风,NaN,5,常温,28.4,19.4
1,2011-01-02 周日,-2,-7,多云,无持续风向微风,NaN,5,常温,28.4,19.4
2,2011-01-03 周一,-2,-6,多云~阴,西北风~北风3-4级~4-5级,NaN,4,常温,28.4,21.2
3,2011-01-04 周二,-2,-9,晴,北风5-6级,NaN,7,常温,28.4,15.8
4,2011-01-05 周三,-2,-10,晴,北风~无持续风向3-4级~微风,NaN,8,常温,28.4,14.0
...,...,...,...,...,...,...,...,...,...,...
4001,2021-12-27 周一,6,-8,晴,西北风1级,56 良,14,常温,42.8,17.6
4002,2021-12-28 周二,6,-5,多云~晴,西北风1级,64 良,11,常温,42.8,23.0
4003,2021-12-29 周三,5,-5,晴,西北风3级,43 优,10,常温,41.0,23.0
4004,2021-12-30 周四,6,-7,晴,西北风3级,38 优,13,常温,42.8,19.4


4. 按条件选择分组分别赋值
按条件先选择数据，然后对这部分数据赋值新列
实例：高低温差大于10度，则认为温差大

In [54]:
# 先创建空列（这是第一种创建新列的方法）
df['wencha_type']=''
df.loc[df['最高温']-df['最低温']>10,'wencha_type']='温差大'
df.loc[df['最高温']-df['最低温']<=10,'wencha_type']='温差正常'

In [55]:
df['wencha_type'].value_counts()

温差正常    2099
温差大     1907
Name: wencha_type, dtype: int64